In [15]:
import numpy as np
from agents.random_agent import random_policy
from citylearn.citylearn import CityLearnEnv
import wandb
import random
import torch
import numpy as np
from collections import namedtuple, deque
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class Constants:
    episodes = 3
    schema_path = './data/citylearn_challenge_2022_phase_1/schema.json'

def action_space_to_dict(aspace):
    """ Only for box space """
    return { "high": aspace.high,
             "low": aspace.low,
             "shape": aspace.shape,
             "dtype": str(aspace.dtype)
    }

def env_reset(env):
    observations = env.reset()
    action_space = env.action_space
    observation_space = env.observation_space
    building_info = env.get_building_information()
    building_info = list(building_info.values())
    action_space_dicts = [action_space_to_dict(asp) for asp in action_space]
    observation_space_dicts = [action_space_to_dict(osp) for osp in observation_space]
    obs_dict = {"action_space": action_space_dicts,
                "observation_space": observation_space_dicts,
                "building_info": building_info,
                "observation": observations }
    return obs_dict

In [17]:
env = CityLearnEnv(schema=Constants.schema_path)

In [18]:
Transition_DDPG = namedtuple('Transition', ('state', 'next_state', 'action', 'reward', 'done'))
class DDPG_Memory(object):
    def __init__(self,capacity):
        """
        Replay buffer for DDPG
        INPUT  : state,next_state,action,reward and done
        OUTPUT : batch of off-polocy experience
        """
        self.memory = deque(maxlen=capacity)
        self.capacity = capacity

    def push(self, state, next_state, action, reward, done):
        self.memory.append(Transition_DDPG(state, next_state, action, reward, done))

    def sample(self, batch_size):
        transitions = random.sample(self.memory, batch_size)
        batch = Transition_DDPG(*zip(*transitions))
        return batch

    def __len__(self):
        return len(self.memory)
